### 📊 Module Topological Metrics Plotter

This script generates **multi-panel scatter plots** for module-level topological metrics computed from each group's microbial interaction network.

#### 🔧 Input
- `[group]_module_analysis.csv` from the `group_network_analysis/` folder, generated in the previous step.

#### 📤 Output
- High-resolution `.png` files saved in `group_topology_plots/`, named as `[group]_topology.png`.

#### 📈 Metrics Visualized
- Nodes, Edges
- Density, Average Degree
- Clustering Coefficient, Diameter
- Average Path Length
- Betweenness and Eigenvector Centrality

#### 🧠 Usage
Each dot represents a module, colored uniquely. The plots help visualize how network structure varies between modules across groups.

> This can be used to identify densely connected or highly central modules, potentially indicating biological relevance.


In [16]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib import cm

# Input and output directories
input_dir = r'..group_network_analysis'  # Directory containing module-level metrics
output_dir = r'..group_topology_plots'  # Output directory for plots
os.makedirs(output_dir, exist_ok=True)

# Get all module topology files
topology_files = [f for f in os.listdir(input_dir) if f.endswith('_module_analysis.csv')]

# Define the plotting function
def plot_topology_metrics(file_path, group_name, output_path):
    # Read topology table
    topology_df = pd.read_csv(file_path)

    # Assign distinct colors to each module
    unique_modules = topology_df["Module"].unique()
    color_palette = cm.get_cmap('plasma', len(unique_modules))
    module_colors = {
        module: color_palette(i / len(unique_modules)) for i, module in enumerate(unique_modules)
    }
    colors = [module_colors[module] for module in topology_df["Module"]]

    # Metrics to visualize
    metrics = [
        "Nodes", "Edges", "Density", "Average Degree", "Diameter",
        "Clustering Coefficient", "Average Path Length",
        "Betweenness Centrality", "Eigenvector Centrality"
    ]

    fig, axes = plt.subplots(3, 3, figsize=(18, 12))
    axes = axes.flatten()

    for i, metric in enumerate(metrics):
        if metric not in topology_df.columns:
            continue
        ax = axes[i]
        ax.scatter(
            topology_df["Module"], 
            topology_df[metric],
            color=colors,
            edgecolor="black",
            s=150
        )
        ax.set_title(f"{metric} Across Modules", fontsize=14)
        ax.set_xlabel("Module", fontsize=12)
        ax.set_ylabel(metric, fontsize=12)
        ax.grid(True, linestyle='--', alpha=0.5)
        ax.set_xticks(topology_df["Module"].unique())

    # Remove empty subplots
    for j in range(len(metrics), len(axes)):
        fig.delaxes(axes[j])

    # Add legend
    legend_elements = [
        Line2D([0], [0], marker='o', color='w', label=f"Module {module}",
               markerfacecolor=module_colors[module], markersize=10, linestyle='None')
        for module in unique_modules
    ]
    fig.legend(
        handles=legend_elements,
        loc="lower right",
        bbox_to_anchor=(0.9, 0.95),
        title="Modules",
        fontsize=10,
        title_fontsize=12
    )

    # Title and save
    plt.suptitle(f"Topological Metrics for Group: {group_name}", fontsize=20)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.savefig(output_path, format='png', dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Topological metrics plot saved for group {group_name}: {output_path}")

# Process each topology file
for topology_file in topology_files:
    group_name = topology_file.split('_module_analysis.csv')[0]
    file_path = os.path.join(input_dir, topology_file)
    output_image_path = os.path.join(output_dir, f"{group_name}_topology.png")

    plot_topology_metrics(file_path, group_name, output_image_path)

print("All topological metric plots generated.")


All topological metric plots generated.
